In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "1"
import numpy as np
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import keras

Using TensorFlow backend.


## VGG16 Internal Architecture

![](vgg16_model.png)

In [2]:
from keras.applications import VGG16    # VGG16 is already prebuild in Keras

## Load the VGG16 Model

In [3]:
# Load the pretrained VGG16 model
# It downloads the model if it is not downloaded
vgg_model = VGG16(weights='imagenet', include_top=True)
vgg_model.summary()







_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
____

## Load the data

In [3]:
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
X_val = np.load('X_val.npy')

y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')
y_val = np.load('y_val.npy')

## Create a output Layer

In [4]:
from keras.layers import Dense

In [5]:
n_classes = y_train.shape[1]

In [6]:
n_classes

97

In [8]:
output = Dense(units=n_classes, activation='softmax')

## Connecting the output with VGG16 model

In [7]:
from keras.models import Model

In [10]:
inp = vgg_model.input
out = output(vgg_model.layers[-2].output)
final_model = Model(inp, out)

In [11]:
final_model

## Freeze the VGG Layers

In [12]:
for layer in final_model.layers[:-1]:
    layer.trainable = False

for layer in final_model.layers[-1:]:
    layer.trainable = True

## Compile the model

In [13]:
final_model.compile(loss='categorical_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])
final_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________

## Scale the dataset

In [8]:
X_train = X_train.astype('float32') / 255.
X_val = X_val.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

## Train the model

In [15]:
history = final_model.fit(X_train, y_train, batch_size=128, epochs=10,
                   validation_data=(X_val, y_val))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 4346 samples, validate on 931 samples
Epoch 1/10
4346/4346 [==============================] - 21s 5ms/step - loss: 4.1645 - acc: 0.1546 - val_loss: 3.1307 - val_acc: 0.3437
Epoch 2/10
4346/4346 [==============================] - 12s 3ms/step - loss: 2.5984 - acc: 0.4370 - val_loss: 2.3337 - val_acc: 0.4801
Epoch 3/10
4346/4346 [==============================] - 12s 3ms/step - loss: 1.9799 - acc: 0.5591 - val_loss: 1.9301 - val_acc: 0.5489
Epoch 4/10
4346/4346 [==============================] - 12s 3ms/step - loss: 1.6272 - acc: 0.6404 - val_loss: 1.6821 - val_acc: 0.6337
Epoch 5/10
4346/4346 [==============================] - 12s 3ms/step - loss: 1.4024 - acc: 0.6815 - val_loss: 1.5540 - val_acc: 0.6412
Epoch 6/10
4346/4346 [==============================] - 12s 3ms/step - loss: 1.2648 - acc: 0.7181 - val_loss: 1.4470 - val_acc: 0.6681
Epoch 7/10
4346/4346 [===========================

## Overfitting!!!!

The model is clearly overfitting. But in that same 10 epochs, the validation accuracy is almost **69%** which is far better than the previous model. But the training accuracy is around **79%**. This clearly means the model is overfitting. So let's add some droupout between the **Fully Connected Dense** layers and train them.

#### First clear the GPU memory (Use only if you use GPU)

In [26]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [9]:
from keras.layers import Dropout

In [10]:
vgg_model = VGG16(weights='imagenet', include_top=True)
fc1 = vgg_model.layers[-3]
fc2 = vgg_model.layers[-2]
output = Dense(units=n_classes, activation='softmax')

In [11]:
dropout1 = Dropout(0.8)
dropout2 = Dropout(0.7)
output = Dense(units=n_classes, activation='softmax')

In [12]:
x = dropout1(fc1.output)
x = fc2(x)
x = dropout2(x)
output = output(x)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
droput_model = Model(input=vgg_model.input, output=output)

/home/antpc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [14]:
droput_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

## Freeze the Conv units in VGG Layers

In [15]:
droput_model.layers[:-5]

In [16]:
for layer in droput_model.layers[:-5]:
    layer.trainable = False

for layer in droput_model.layers[-5:]:
    layer.trainable = True

In [17]:
droput_model.compile(loss='categorical_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy'])

In [18]:
history = droput_model.fit(X_train, y_train, batch_size=128, epochs=100,
                   validation_data=(X_val, y_val))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 4346 samples, validate on 931 samples
Epoch 1/100
4346/4346 [==============================] - 25s 6ms/step - loss: 5.3268 - acc: 0.0161 - val_loss: 4.5675 - val_acc: 0.0462
Epoch 2/100
4346/4346 [==============================] - 13s 3ms/step - loss: 4.6275 - acc: 0.0249 - val_loss: 4.5397 - val_acc: 0.0322
Epoch 3/100
4346/4346 [==============================] - 13s 3ms/step - loss: 4.5436 - acc: 0.0460 - val_loss: 4.4454 - val_acc: 0.0430
Epoch 4/100
4346/4346 [==============================] - 13s 3ms/step - loss: 4.4747 - acc: 0.0529 - val_loss: 4.2799 - val_acc: 0.0827
Epoch 5/100
4346/4346 [==============================] - 13s 3ms/step - loss: 4.4021 - acc: 0.0637 - val_loss: 4.1793 - val_acc: 0.0816
Epoch 6/100
4346/4346 [==============================] - 13s 3ms/step - loss: 4.2990 - acc: 0.0874 - val_loss: 4.0267 - val_acc: 0.1547
Epoch 7/100
4346/4346 [====================

4346/4346 [==============================] - 13s 3ms/step - loss: 2.4090 - acc: 0.4584 - val_loss: 2.1093 - val_acc: 0.4984
Epoch 59/100
4346/4346 [==============================] - 13s 3ms/step - loss: 2.4119 - acc: 0.4781 - val_loss: 2.2185 - val_acc: 0.4758
Epoch 60/100
4346/4346 [==============================] - 13s 3ms/step - loss: 2.3932 - acc: 0.4669 - val_loss: 2.0653 - val_acc: 0.5134
Epoch 61/100
4346/4346 [==============================] - 13s 3ms/step - loss: 2.3115 - acc: 0.4814 - val_loss: 2.0484 - val_acc: 0.5081
Epoch 62/100
4346/4346 [==============================] - 13s 3ms/step - loss: 2.3447 - acc: 0.4823 - val_loss: 2.0981 - val_acc: 0.4930
Epoch 63/100
4346/4346 [==============================] - 13s 3ms/step - loss: 2.2799 - acc: 0.4867 - val_loss: 2.0002 - val_acc: 0.5145
Epoch 64/100
4346/4346 [==============================] - 13s 3ms/step - loss: 2.3745 - acc: 0.4830 - val_loss: 2.0127 - val_acc: 0.5113
Epoch 65/100
4346/4346 [==============================

In [19]:
loss, accuracy = droput_model.evaluate(X_test, y_test)

932/932 [==============================] - 4s 4ms/step


In [20]:
loss

1.7852234886439573

In [21]:
accuracy

0.5718884120171673